# Welcome to colab's MMD Auto Trace! (Execution)

# MMD Auto Trace Kit preparation

This notebook prepares and executes MMD Auto Trace.

Click ">" on the top left of the screen. The table of contents opens.

![目次](https://drive.google.com/uc?export=view&id=1x8AdFNmsIQPrtYptBf_NXPRNBJF8ON8z)

Check the notebooks from top to bottom and perform the following steps one by one.

- **「Change runtime to GPU」**
  - Make sure the runtime has changed to GPU
  - Please check the introductory section for how to change
- **「Cooperation with Google Drive」**
  - Make sure you can work with Google Drive
  - Please check the introductory version for how to cooperate
- **「Preparation batch execution」**
    - Run all the cells in the preparation section
      - In this process, all programs and data required for MMD Auto Trace are created on colab.
      - It takes about 40 to 60 minutes.
- **「MMD Auto Trace Kit execution」**
  - Execute cells in the execution section one by one from the top
    - Specifying Trace Source Video
    - Trace parameter settings
    - Trace processing execution
    - Depending on the number of people, it will take approximately 50 to 60 minutes with 6000 frames.


## Change runtime to GPU

Select "Runtime"> "Change Runtime Type"> "GPU" in the header.

If you can change it, please execute the cell below.

In [0]:
! nvcc --version
! nvidia-smi

**【OK】**

It is a success if it is displayed as follows.

![GPU変更成功](https://drive.google.com/uc?export=view&id=17CG697kiTLkwVOdH1wg2W3MSB-hyi9u5)

---

**【NG】**

If it is displayed as shown below, the runtime change has failed, so check the introductory version again and change the runtime.

![GPU切り替え失敗](https://drive.google.com/uc?export=view&id=1tufSuT7ocWxv3HkrmA5kwlemhu0gv6Je)

## Cooperation with Google Drive

Works with the Google Drive `autotrace` folder.

Please execute the cell below.

In [0]:
from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "Contents of [autotrace] folder -----------"
! ls -l "$base_path"
! echo "--------------------"

## Preparation batch execution

Here, the cells in the preparation section are executed collectively.

Select "Execute MMD Auto Trace Kit" in the table of contents.

![実行選択](https://drive.google.com/uc?export=view&id=17HmndobtAh5ak4NR0g7E4cElkDEml6tw)

If you select "Runtime"> "Perform cells before" in the header, all cells in the preparation section will be executed sequentially.

![すべてのセルを実行](https://drive.google.com/uc?export=view&id=1Y6WJ8EHMKmF9X3M_hH9TZE5VHea1swWn)

---

** 【OK】**

The following is output at the bottom of the screen, and it is complete.

![処理成功](https://drive.google.com/uc?export=view&id=1D21xezv6QN0RQF5ZU_LR7PRnOk0Dw4Sc)

It takes about 40 to 60 minutes.

---

**【NG】**

![処理失敗](https://drive.google.com/uc?export=view&id=1t-immeF3Ji1_GBNatZOG1C07j42de4Rq)

It is a failure if "No such file or directory" is output on the last line.

If you don't know the solution, share your notebook.

### Environmental setting

In [0]:
# 処理時間計測のための開始時間
import time
start_time = time.time()

In [0]:
# Openposeバージョンタグ
ver_openpose = "v1.4.0"

In [0]:
# MMD自動トレースキットバージョンタグ
ver_tag = "ver1.01"

### cmake

In [0]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

### Openpose

In [0]:
# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python2 libs
! sudo apt-get --assume-yes install python-setuptools python-dev build-essential
! sudo easy_install pip
! sudo -H pip install --upgrade numpy protobuf opencv-python
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [0]:
#  Openpose の clone
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 
! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git     

In [0]:
#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

In [0]:
# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc` # example demo usage

### FCRN-DepthPrediction-vmd

In [0]:
# FCRN-DepthPrediction-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/FCRN-DepthPrediction-vmd.git

In [0]:
# FCRN-DepthPrediction-vmd の モデルデータDL

# モデルデータ の導入
! mkdir -p ./FCRN-DepthPrediction-vmd/tensorflow/data

# モデルデータのダウンロード
! cd  ./FCRN-DepthPrediction-vmd/tensorflow/data && wget -c "http://campar.in.tum.de/files/rupprecht/depthpred/NYU_FCRN-checkpoint.zip" && unzip NYU_FCRN-checkpoint.zip

### 3d-pose-baseline-vmd

In [0]:
# 3d-pose-baseline-vmd の clone
! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/3d-pose-baseline-vmd.git

In [0]:
# 3d-pose-baseline-vmd の Human3.6MモデルデータDL

# Human3.6Mモデルデータ の導入
! mkdir -p ./3d-pose-baseline-vmd/data

# Human3.6Mモデルデータのダウンロード
! cd  ./3d-pose-baseline-vmd/data && wget -c "https://www.dropbox.com/s/e35qv3n6zlkouki/h36m.zip" && unzip h36m.zip

In [0]:
# 3d-pose-baseline-vmd の 学習データDL

# 3d-pose-baseline用学習データ の導入
! mkdir -p ./3d-pose-baseline-vmd/experiments

# 3d-pose-baseline用学習データのダウンロード
file_id = "1v7ccpms3ZR8ExWWwVfcSpjMsGscDYH7_"
file_name = "experiments.zip"
! cd  ./3d-pose-baseline-vmd && curl -sc ./cookie "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
code = "$(awk '/_warning_/ {print $NF}' ./cookie)"  
! cd  ./3d-pose-baseline-vmd && curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$code&id=$file_id" -o "$file_name"
! cd  ./3d-pose-baseline-vmd && unzip experiments.zip

### VMD-3d-pose-baseline-multi

In [0]:
# VMD-3d-pose-baseline-multi の clone

! git clone  --depth 1 -b "$ver_tag" https://github.com/miu200521358/VMD-3d-pose-baseline-multi.git

In [0]:
# VMD-3d-pose-baseline-multi のライブラリ

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools

### Preparation result confirmation

In [0]:
# サンプルの実行確認
! cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json ./output/ --display 0  --write_video ./output/openpose.avi

In [0]:
import time

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■すべての処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo "Openpose実行結果"

! ls -l ./openpose/output/openpose.avi

# Execute MMD Auto Trace Kit

## Installation complete confirmation

From here, we will actually execute the kit.

In "Run previous cell", did all the installation have been performed?

For more information, check the Getting Started section.

Once you're ready, run the cell below to see if the installation is complete.

In [0]:
!ls -l ./openpose/README.md
!ls -l ./FCRN-DepthPrediction-vmd/README.md
!ls -l ./3d-pose-baseline-vmd/README.md
!ls -l ./VMD-3d-pose-baseline-multi/README.md
!ls -l ./openpose/output/openpose.avi

**【OK】**

If the file name and file size are displayed as shown below, installation is complete.

Please proceed to the input video file upload.

![インストール成功](https://drive.google.com/uc?export=view&id=1l13A2iF9oTpGcZSe9q8k7JyDyiABxOQT)

---

**【NG】**

If "No such file or directory" is displayed as shown below, installation fails.

![インストール失敗](https://drive.google.com/uc?export=view&id=1LuKoSMwFOzFg8NguFxqAtmQy9B1_KMXr)

Return to the top of this notebook and try again from the beginning.

If the installation fails after three attempts, share your notebook.

## Input video file upload

Prepare the video file you want to process.

 - The file name should be **only single-byte alphanumeric characters**. opencv can not read double-byte characters.
 - Please put it under the ** autotrace ** folder of Google Drive.
 - FPS should be ** 30fps ** or ** 60fps **.
 - The size should be ** 1280x720 **.
 - If the size or fps is not as specified, the program will re-encode. (Fps will be 30)
 - ** Overwriting or updating files on the Goole drive after mounting is not recognized correctly. ** Please upload a new file with a new name and process it.
 - When tracing the number of people, if people disappear from the screen, the order can not be acquired and the traces of the unplaced persons will be incorrect. Make sure that all the members of the person you want to acquire are displayed as much as possible.
 - Once the upload is complete, please execute the cells below sequentially.

In [0]:
#@markdown ### 【O】Input video file
#@markdown Enter the name of the video file to be analyzed.
#@markdown If the width is not 1280 or if the frame rate is not 30 fps, re-encoding is performed.
input_video_name = "input.mp4"  #@param {type: "string"}

from google.colab import drive
import os
import cv2
import datetime

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/autotrace"

! echo "autotraceフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

# 入力動画ファイル
input_video = base_path + "/"+ input_video_name

print("ファイル名: ", os.path.basename(input_video))
print("ファイルサイズ: ", os.path.getsize(input_video))


video = cv2.VideoCapture(input_video)
# 幅
W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
# 高さ
H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
# 総フレーム数
count = video.get(cv2.CAP_PROP_FRAME_COUNT)
# fps
fps = video.get(cv2.CAP_PROP_FPS)

print("横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))



width = 1280
height = 720

if W != 1280 or (fps != 30 and fps != 60):
    print("大きさもしくはfpsが処理対象外のため、再エンコードします: "+ input_video)
    
    # 縮尺
    scale = width / W
    
    # 高さ
    height = int(H * scale)

    # 出力ファイルパス
    out_name = 'recode_{0}.mp4'.format("{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now()))
    out_path = '{0}/{1}'.format(base_path, out_name)
    
    try:
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        out = cv2.VideoWriter(out_path, fourcc, 30.0, (width, height), True)
        # 入力ファイル
        cap = cv2.VideoCapture(input_video)

        while(cap.isOpened()):
            # 動画から1枚キャプチャして読み込む
            flag, frame = cap.read()  # Capture frame-by-frame

            # 動画が終わっていたら終了
            if flag == False:
                break

            # 縮小
            output_frame = cv2.resize(frame, (width, height))

            # 出力
            out.write(output_frame)

        # 終わったら開放
        out.release()
    except Exception as e:
        print("再エンコード失敗", e)

    cap.release()
    cv2.destroyAllWindows()
    
    print('MMD入力用MP4ファイル再生成', out_path)
    input_video_name = out_name

    # 入力動画ファイル再設定
    input_video = base_path + "/"+ input_video_name
    
    video = cv2.VideoCapture(input_video)
    # 幅
    W = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    # 高さ
    H = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    # 総フレーム数
    count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # fps
    fps = video.get(cv2.CAP_PROP_FPS)

    print("【再チェック】横: {0}, 縦: {1}, フレーム数: {2}, fps: {3}".format(W, H, count, fps))

    
!echo "入力映像ファイルは" "$input_video_name" "です。"
!echo ""
!echo "問題なければ、次へ進んで下さい。"

It is successful if the file name can be obtained at the end of the process.

---
**【OK】**

If the file size and fps are as specified, the input file is handled as it is.

![OK](https://drive.google.com/uc?export=view&id=1lvOhCAj99_NUNDb-wfRAxeu-o0Exth7v)

----
**【Re-encoding】**

If the file size and fps are not as specified, the re-encoded mp4 file is treated as an input file.

![再エンコード](https://drive.google.com/uc?export=view&id=1xEiy-pdeHWQpt4CLZePg9YT1_7U8bEqz)


## Parameter setting

Please set the parameters.

 - 【O】… Parameters used by Openpose
 - 【F】… Parameters used by FCRN-DepthPrediction-vmd
 - 【V】… Parameters used by VMD-3d-pose-baseline-multi

In [0]:
#@markdown Enter the parameters for tracing the image and execute the cell.

#@markdown --- 

#@markdown ### 【O】Maximum number of people in the video
#@markdown Please enter the number of people you want to get from the video.
#@markdown Please process the video data as much as possible for this number of people.
number_people_max = 1  #@param {type: "number"}

#@markdown --- 

#@markdown ### 【O】Frame number to start analysis
#@markdown Enter the frame number to start analysis. (0 beginning)
#@markdown If the human body can not be traced accurately, for example, if the logo is displayed first, specify the first frame in which all the members appear in the image.
frame_first = 0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【F】Frame number to finish analysis
#@markdown Please enter the frame number to finish the analysis. (0 beginning)
#@markdown When you adjust the reverse or order in "FCRN-DepthPrediction-vmd", you can finish the process and see the result without outputting to the end.
#@markdown If the default value is "-1", analysis is performed to the end.
end_frame_no = -1  #@param {type: "number"}

#@markdown --- 

#@markdown ### 【F】Reverse specification list
#@markdown Specify the frame number (0 starting) that is inverted by Openpose by mistake, the person INDEX order, and the contents of the inversion.
#@markdown In the order that Openpose recognizes at 0F, INDEX is assigned as 0, 1, ....
#@markdown Format: [{frame number}: Person who wants to specify reverse INDEX, {reverse content}]
#@markdown {reverse content}: R: Whole body inversion, U: Upper body inversion, L: Lower body inversion, N: No inversion
#@markdown ex）[10:1,R]　…　The whole person flips the first person in the 10th frame.
#@markdown Since the contents are output in the above format in message.log when inverted output, please refer to that.
#@markdown As in [10:1,R][30:0,U], multiple items can be specified in parentheses.
reverse_specific = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【F】Ordered list
#@markdown In the multi-person trace, please specify the person INDEX order after crossing.
#@markdown In the case of a one-person trace, it is OK to leave it blank.
#@markdown In the order that Openpose recognizes at 0F, INDEX is assigned as 0, 1, ....
#@markdown Format: [{frame number}: index of first estimated person, index of first estimated person, ...]
#@markdown 例）[10:1,0]　…　The order of the 10th frame is rearranged in the order of the first person from the left and the zeroth person.
#@markdown The order in which messages are output in message.log is left in the above format, so please refer to it.
#@markdown As in [10:1,0][30:0,1], multiple items can be specified in parentheses.
#@markdown Also, in output_XXX.avi, colors are assigned to people in the estimated order. The right half of the body is red and the left half is the following color.
#@markdown 0: green, 1: blue, 2: white, 3: yellow, 4: peach, 5: light blue, 6: dark green, 7: dark blue, 8: gray, 9: dark yellow, 10: dark peach, 11: dark light blue
order_specific = ""  #@param {type: "string"}

#@markdown --- 

#@markdown ### 【V】Bone structure CSV file
#@markdown Select or enter the path of the bone structure CSV file of the trace target model.
#@markdown You can select "Animasa-Miku" and "Animasa-Miku semi-standard", or you can input a bone structure CSV file of any model.
#@markdown If you want to input any model bone structure CSV file, please upload the csv file to the "autotrace" folder of Google Drive.
#@markdown And please enter like「/gdrive/My Drive/autotrace/[csv file name]」
born_model_csv = "born/animasa_miku_born.csv" #@param ["born/animasa_miku_born.csv", "born/animasa_miku_semi_standard_born.csv"] {allow-input: true}


#@markdown --- 

#@markdown ### 【V】Whether to output with IK
#@markdown Output the foot of trace data as IK, or select yes or no.
#@markdown If you enter no, output with FK
ik_flag = "yes"  #@param ['yes', 'no']
is_ik = 1 if ik_flag == "yes" else 0

#@markdown ---

#@markdown ### 【V】Heel position correction
#@markdown Please input the Y axis correction value of the heel with a numerical value (decimal possible).
#@markdown Entering a negative value approaches the ground, entering a positive value moves away from the ground.
#@markdown Although it corrects automatically to some extent automatically, if you can not correct it, please set it.
heel_position = 0.0  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】Center Z moving magnification
#@markdown Please enter the magnification multiplied by the center Z movement with a numerical value (decimal possible).
#@markdown The smaller the value, the smaller the width of the center Z movement.
#@markdown When 0 is input, center Z axis movement is not performed.
center_z_scale = 5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】Smoothing frequency
#@markdown Specify the degree of motion smoothing
#@markdown Please enter only an integer of 1 or more.
#@markdown The larger the frequency, the smoother it is. (The behavior will be smaller instead)
smooth_times = 1  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】Movement key thinning amount
#@markdown Specify the amount of movement to be used for decimation of movement key (IK, center) with numerical value (decimal possible)
#@markdown When there is a movement within the specified range, it is thinned out.
#@markdown When moving thinning amount is set to 0, thinning is not performed.
threshold_pos = 0.5  #@param {type: "number"}

#@markdown ---

#@markdown ### 【V】Rotating Key Culling Angle
#@markdown Specify the angle (decimal possible from 0 to 180 degrees) to be used for decimating rotation keys
#@markdown It will be thinned out if there is a rotation within the specified angle.
threshold_rot = 3  #@param {type: "number"}



!echo 【O】Maximum number of people in the video: "$number_people_max"
!echo 【O】Frame number to start analysis: "$frame_first"
!echo 【F】Frame number to finish analysis: "$end_frame_no"
!echo 【F】Reverse specification list: "$reverse_specific"
!echo 【F】Ordered list: "$order_specific"
!echo 【V】Bone structure CSV file: "$born_model_csv"
!echo 【V】Whether to output with IK: "$ik_flag"
!echo 【V】Heel position correction: "$heel_position"
!echo 【V】Center Z moving magnification: "$center_z_scale"
!echo 【V】Smoothing frequency: "$smooth_times"
!echo 【V】Movement key thinning amount: "$threshold_pos"
!echo 【V】Rotating Key Culling Angle: "$threshold_rot"

!echo ""
!echo If the above is correct, please proceed to the next.

## Auto Trace execution (all execution)

In [0]:
#@markdown Please execute this cell after completing all the forms.
#@markdown Processing is performed in the following order.

#@markdown 1. Openpose（Video→2D）
#@markdown 2. FCRN-DepthPrediction-vmd（Depth estimation）
#@markdown 3. 3d-pose-baseline-vmd（2D→3D）
#@markdown 4. VMD-3d-pose-baseline-multi（3D→VMD）

#@markdown Depending on the number of traces, it takes about 50 to 60 minutes in 6000 frames.
#@markdown When Openpose starts, it looks like it has stopped moving for a while with its elongated square.
#@markdown If the playback button is spinning around, processing is taking place behind the scenes, so please wait without doing anything.
#@markdown If the vmd file has not been generated, the contents of pos.txt are empty, there is only error.txt, etc., first check the contents of error.txt, and check and execute the "If an error occurs" section please do it.

import time
import datetime
import cv2
from google.colab import drive

start_time = time.time()

# 出力フォルダ削除
if os.path.exists("./output"):
    !rm -r ./output

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"
! mkdir -p "$output_json"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo Openpose
! echo ------------------------------------------

# Openpose実行
! cd openpose/ && ./build/examples/openpose/openpose.bin --video "$input_video" --display 0 --model_pose COCO --write_json "$output_json" --write_video "$output_openpose_avi" --frame_first "$frame_first" --number_people_max "$number_people_max"

! echo ------------------------------------------
! echo FCRN-DepthPrediction-vmd
! echo ------------------------------------------

! cd FCRN-DepthPrediction-vmd && python tensorflow/predict_video.py --model_path tensorflow/data/NYU_FCRN.ckpt --video_path "$input_video" --json_path "$output_json" --interval 10 --reverse_specific "$reverse_specific" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max" --end_frame_no "$end_frame_no"
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_specific.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    for i in range(1, number_people_max+1):
        ! echo ------------------------------------------
        ! echo 3d-pose-baseline-vmd ["$i"]
        ! echo ------------------------------------------

        target_name = "_" + now_str + "_idx0" + str(i)
        target_dir = output_json + target_name

        !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

        ! echo ------------------------------------------
        ! echo VMD-3d-pose-baseline-multi ["$i"]
        ! echo ------------------------------------------

        ! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"

        # INDEX別結果コピー
        idx_dir_path = drive_dir_path + "/idx0" + str(i)
        ! mkdir -p "$idx_dir_path"
        ! cp "$target_dir"/*.vmd "$idx_dir_path"
        ! cp "$target_dir"/pos.txt "$idx_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■すべての処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"
    ! ls -l "$drive_dir_path"

## Auto Trace execution (partial execution)

**If you do not need to execute again after completing「Auto Trace execution (all execution)」, this is the end.** 

If you want to make any corrections, trace again as follows.

1. If you want to change the trace source video
  - Upload new videos to ** autotrace ** folder on Google Drive with ** new file name **
 - Execute the cell of ** "Input video file upload" **
 - Execute the cell of ** "Parameter Setting" **
 - Execute the cell of ** "Automatic trace execution (all execution)" **  
  
2. When you want to change the parameter of【O】
 - Change the【O】value of ** "Parameter setting" **
 - Execute the ** "Parameter Setting" ** cell
 - Execute the cell of ** "Automatic trace execution (all execution)" **

3. When you want to change the parameter of【F】
 - Change the【F】value of ** "Parameter setting" **
 - Execute the ** "Parameter Setting" ** cell
 - Execute the cell of ** "A) Automatic trace rerun (depth estimation)" **
 - When satisfied with the parameters of【F】
       - Execute the cell of ** "B) Auto Trace rerun (2D → 3D)" **
       - Execute the cell of ** "C) Auto Trace rerun (3D → VMD)" ** 
 
4. When you want to change the parameter of【V】
 - Change the【V】value of ** "Parameter setting" **
 - Execute the ** "Parameter Setting" ** cell
 - Execute the cell of ** "C) Auto Trace rerun (3D → VMD)" ** 


### A) Auto trace rerun (depth estimation)

In [0]:
import time
import datetime
import cv2
import os
from google.colab import drive

start_time = time.time()

# 処理日時
now_str = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

! echo ------------------------------------------
! echo FCRN-DepthPrediction-vmd
! echo ------------------------------------------

! cd FCRN-DepthPrediction-vmd && python tensorflow/predict_video.py --model_path tensorflow/data/NYU_FCRN.ckpt --video_path "$input_video" --json_path "$output_json" --interval 10 --reverse_specific "$reverse_specific" --order_specific "$order_specific" --verbose 1 --now "$now_str" --avi_output "yes"  --number_people_max "$number_people_max" --end_frame_no "$end_frame_no"
    
# 深度結果コピー
depth_dir_path =  output_json + "_" + now_str + "_depth"

if os.path.exists( depth_dir_path + "/error.txt"):
    
    # エラー発生
    ! cp "$depth_dir_path"/error.txt "$drive_dir_path"

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■エラーが発生したため、処理を中断しました。"
    ! echo "■■"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo "$drive_dir_path" "の error.txt の中身を確認してください。"

else:
    
    ! cp "$depth_dir_path"/*.avi "$drive_dir_path"
    ! cp "$depth_dir_path"/message.log "$drive_dir_path"
    ! cp "$depth_dir_path"/reverse_specific.txt "$drive_dir_path"
    ! cp "$depth_dir_path"/order_specific.txt "$drive_dir_path"

    # Googleドライブ再マウント
    drive.mount('/gdrive')

    elapsed_time = (time.time() - start_time) / 60

    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
    ! echo "■■【深度推定】の処理が終了しました"
    ! echo "■■"
    ! echo "■■処理にかかった時間：" "$elapsed_time" "分"
    ! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

    ! echo ""
    ! echo "MMD自動トレース実行結果"

    ! echo "$drive_dir_path"    
    ! ls -l "$drive_dir_path"

### B) Auto Trace rerun (2D → 3D)

In [0]:
import time
import datetime
import cv2
import os
from google.colab import drive

start_time = time.time()

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"

for i in range(1, number_people_max+1):
    ! echo ------------------------------------------
    ! echo 3d-pose-baseline-vmd ["$i"]
    ! echo ------------------------------------------

    target_name = "_" + now_str + "_idx0" + str(i)
    target_dir = output_json + target_name

    !cd ./3d-pose-baseline-vmd && python src/openpose_3dpose_sandbox_vmd.py --camera_frame --residual --batch_norm --dropout 0.5 --max_norm --evaluateActionWise --use_sh --epochs 200 --load 4874200 --gif_fps 30 --verbose 1 --openpose "$target_dir" --person_idx 1    

    # INDEX別結果コピー
    idx_dir_path = drive_dir_path + "/idx0" + str(i)
    ! mkdir -p "$idx_dir_path"
    ! cp "$target_dir"/pos.txt "$idx_dir_path"

# Googleドライブ再マウント
drive.mount('/gdrive')

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■【2D→3D】の処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo ""
! echo "MMD自動トレース実行結果"

! echo "$drive_dir_path"    
! ls -l "$drive_dir_path"

### C) Auto Trace rerun (3D → VMD)

In [0]:
import time
import datetime
import cv2
from google.colab import drive

start_time = time.time()

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
drive_base_dir = "/gdrive/My Drive/autotrace"

output_json = "/content/output/json"
output_openpose_avi = "/content/output/openpose.avi"

# 出力用Googleドライブフォルダ
drive_dir_path = drive_base_dir + "/" + now_str 
! mkdir -p "$drive_dir_path"


for i in range(1, number_people_max+1):

    ! echo ------------------------------------------
    ! echo VMD-3d-pose-baseline-multi ["$i"]
    ! echo ------------------------------------------

    ! cd ./VMD-3d-pose-baseline-multi && python applications/pos2vmd_multi.py -v 2 -t "$target_dir" -b "$born_model_csv" -c 30 -z "$center_z_scale" -s "$smooth_times" -p "$threshold_pos" -r "$threshold_rot" -k "$is_ik" -e "$heel_position"

    # INDEX別結果コピー
    idx_dir_path = drive_dir_path + "/idx0" + str(i)
    ! mkdir -p "$idx_dir_path"
    ! cp "$target_dir"/*.vmd "$idx_dir_path"

# Googleドライブ再マウント
drive.mount('/gdrive')

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■■【3D→VMD】の処理が終了しました"
! echo "■■"
! echo "■■処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"

! echo ""
! echo "MMD自動トレース実行結果"

! echo "$drive_dir_path"
! ls -l "$drive_dir_path"

# If an error occurs

If an error occurs, and if the vmd file has not been generated, execute this section one by one from the top.

If you still have problems, follow the introductory instructions to share a copy of your notebook.

## 1. If the number of people in the first frame read by Openpose does not appear

If the error.txt says "There is no data for the number of people in the first frame", the reason is that there are no data for the first frame read by Openpose.

Please execute the cell below.

In [0]:
!find output/json/ -name "*.json" | sort | head -n 1 | xargs ls -l
!find output/json/ -name "*.json" | sort | head -n 1 | xargs cat

【Status】

As shown below, if there is no data after people, no human data can be obtained at frame 0.

![結果なし](https://drive.google.com/uc?export=view&id=1osssF0NCWply6J0-zPIhN2wm1gT0o6Io)

【Solution】

Specify the first frame in which a person appears in "【O】Frame number to start analysis".

The top 30 Openpose result files will be displayed when you execute the cell "30 list of frames read by Openpose".

![先頭30件](https://drive.google.com/uc?export=view&id=1lxP78w4NIbQSKWhpfbynCjDKOgmcNp0o)

JSON files without personal data have very small file size. (In the case of the figure, the 0th frame is no person data)

The data size is about 500 bytes (0.5 KB) for one person's data.

Please refer to this and decide the top frame number. There is no need to edit or re-upload the original video.

When the first frame number is determined, enter the number in "【O】Frame number to start analysis" in the "Parameter setting" section, and select "Parameter setting"> "Auto Trace execution (all execution)" in this order. Please practice.


In the case of a multi-person trace, it is necessary for all the members to be shown in the 0th frame (【O】Frame number to start analysis).

The file size will also increase by the number of people, so please make a guide.

## 2. 30 list of frames read by Openpose

In [0]:
ls -l output/json/*.json | head -n 30

## 3. When file is not added to Google Drive

If neither error.txt nor vmd is output, check the cell output.

The output itself is successful if there is a final list of file names.

However, I have confirmed the case where the data is not reflected even though the linkage with Google Drive has been completed.

The details are under investigation, but please download the original data on the cloud for the time being.

1. Click the "File" column next to the table of contents
2. Click "Update" in the header
3. Open "output ＞ json"
4. xxx_depth ＞ output_XXX.avi …　Background AVI(MMD)
5. xxx_idxXX ＞ output_XXX.vmd　…　Motion data(MMD)
6. xxx_idxXX ＞ pos.txt　…　3D joint position data(Unity)
7. If you trace multiple people, there are multiple idx.

![クラウドデータ](https://drive.google.com/uc?export=view&id=1fArRyRdfs1kBLaLTpdkdJ-MYwHNe-UUq)


## 4. If you want to redo everything

If you have trouble preparing everything and want to start over, reset the runtime.

Header >  Runtime > Reset All Runtimes

![リセット](https://drive.google.com/uc?export=view&id=1HNOEDju8R5pTZseJ0FCOnFVDRI9ruuLC)

A confirmation dialog will appear, so please proceed with OK.

# TIPS

In addition to the above, I will write things that can be helpful to you as I think.

## Recommended work order

I work in the following order.

1.  Execute the cell by entering the number of people you want to trace from the trace source video in  ** “【O】Maximum number of people in the video” **
2.  Execute the cell of ** "Auto Trace execution (all execution)" **
3. If the result is an error, execute ** "Is the number of people in the first frame of Openpose read" ** in ** "If an error occurs" ** and check the frame in which the person appears Do. Move to 6 if successful.
4.  Enter the frame number found in step 3 in ** "【O】Frame number to start analysis" ** 
5. Execute the cell of  ** "Parameter setting" ** 
6.  Execute the cell of ** "A) Auto trace rerun (depth estimation)" ** 
7. If the trace can not be recognized in multiple people's trace, specify the order in  ** "【F】Ordered list" **  while looking at message.log
8.  Execute the cell of ** "Parameter Setting" ** 
9. Execute the cell of ** "A) Auto trace rerun (depth estimation)" **
10. Repeat 7 to 9 until you are satisfied with the designated order
11. If there is an unintended rotation, specify and set the correct inversion status of the corresponding frame in  ** "【F】Reverse specification list" **  while looking at message.log

  - ※ If the order is changed in the minor lightness people trace, the reverse specified person INDEX will also change, so it is better to execute after the order specification is finished.
  
12. Execute the cell of  ** "parameter setting" **  
13.  Execute the cell of ** "A) Auto trace rerun (depth estimation)" **
14. Repeat 11 to 13 until you are satisfied
15. Execute ** "B) Auto Trace rerun (2D → 3D)" **   when the specification of replacement or rotation is completed.
16.  Adjust the [V] value of “Parameter setting” 
17. Execute the cell of  ** "parameter setting" **  
18.  Execute the cell of ** "C) Auto Trace rerun (3D → VMD)" **  . Even in the case of a multi-person trace, output for the number of people is performed at one time.
19. Repeat 15-18 until you are satisfied

good luck!

## An easy to trace video

 - It is a fixed camera
 - The joints are clearly visible
    - Human body with a hard to see joints such as long skirts and Japanese clothes is not good at analysis
    - When the background is a color similar to a person, the shadow is dark, etc., the trace is often mistaken.
    - The accuracy is better if the wrist and ankle are seen
    - Accuracy is reduced when it is difficult to distinguish between left and right with black trousers etc.
 - Front facing in the first frame
   - I can not get the data of the beginning clearly when I look at the back or side (I often fix it once I look at the front)
 - The joints of the whole body can be identified in the first frame
   - If you are hiding somewhere, the accuracy will drop
 - Head up, foot down
   - If your leg rises high due to a handstand or kick, your foot is misrecognized as a hand. (Especially when the joints at the foot of the foot are above the neck)


## Task

 - Joint whose rotation can not be estimated
    - Head
    - Wrist
    - Finger
 - Depth of human body (center Z)

# License

When publishing or distributing the results of "MMD Auto Trace", please be sure to confirm the license. The same is true for Unity.

I am very grateful if you can enter a license. 

[MMD Auto Trace License(Japanese)](https://ch.nicovideo.jp/miu200521358/blomaga/ar1686913)